特徴量作成にあたり参考にした記事：<br>
https://bstatsplus.com/glossary<br>
https://www.nogawanogawa.com/entry/basketball_analysis<br>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
import seaborn as sns; sns.set()

# pd.get_option("display.max_columns")
# pd.set_option('display.max_columns', 100)

DATA_DIR = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/MDataFiles_Stage2'
STAGE_1 = True

# シード情報特徴量・qualityの作成

In [ ]:
regular_results = pd.read_csv(DATA_DIR+'/MRegularSeasonCompactResults.csv')
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'         
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    output = pd.concat([df, dfswap]).sort_index().reset_index(drop=True)
    
    return output

regular_results = prepare_data(regular_results)
# regular_results = regular_results.loc[:, ['Season', 'T1_TeamID', 'T2_TeamID', 'location', 'NumOT']]
# regular_results = regular_results.loc[regular_results['Season'] >= 2015]
regular_results['T1_win'] = 0
regular_results['T2_win'] = 0
regular_results.loc[regular_results['T1_Score'] > regular_results['T2_Score'], 'T1_win'] = 1
regular_results.loc[regular_results['T1_Score'] < regular_results['T2_Score'], 'T2_win'] = 1
regular_results

In [ ]:
# convert to str, so the model would treat TeamID them as factors
# 重回帰分析によって各シーズンのチーム毎のパフォーマンスを予測する処理
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(str)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(str)

# make it a binary task
regular_results['win'] = np.where(regular_results['T1_Score']>regular_results['T2_Score'], 1, 0)

def team_quality(season):
    """
    Calculate team quality for each season seperately. 
    Team strength changes from season to season (students playing change!)
    So pooling everything would be bad approach!
    """
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_results.loc[regular_results.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    # extracting parameters from glm
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','beta']
    quality['Season'] = season
    # taking exp due to binomial model being used
    quality['quality'] = np.exp(quality['beta'])
    # only interested in glm parameters with T1_, as T2_ should be mirroring T1_ ones
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

%%time
team_quality = pd.concat([
                          team_quality(2015),
                          team_quality(2016),
                          team_quality(2017),
                          team_quality(2018),
                          team_quality(2019),
                          team_quality(2020),
                          team_quality(2021)
                          ]).reset_index(drop=True)

team_quality

In [ ]:
# 各シードに配置された時の勝率、パワーランク作成
tourney_results = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')
seeds = pd.read_csv(DATA_DIR+'/MNCAATourneySeeds.csv')

tourney_results = prepare_data(tourney_results)

team_quality_copy = team_quality.copy()

team_quality_T1 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T1.columns = ['T1_TeamID','Season','T1_quality']
team_quality_T2 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T2.columns = ['T2_TeamID','Season','T2_quality']

tourney_results['T1_TeamID'] = tourney_results['T1_TeamID'].astype(int)
tourney_results['T2_TeamID'] = tourney_results['T2_TeamID'].astype(int)
tourney_results = tourney_results.merge(team_quality_T1, on = ['T1_TeamID','Season'], how = 'left')
tourney_results = tourney_results.merge(team_quality_T2, on = ['T2_TeamID','Season'], how = 'left')

tourney_results = tourney_results.loc[tourney_results['DayNum'] >= 136].reset_index(drop=True)
tourney_results = tourney_results.loc[tourney_results['Season'] < 2015].reset_index(drop=True)

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

seeds_T1 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T2 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed','T1_division']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed','T2_division']

tourney_results = tourney_results.merge(seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_results = tourney_results.merge(seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_results['T1_powerrank'] = tourney_results.groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False).astype(int)
tourney_results['T2_powerrank'] = tourney_results.groupby(['Season','T2_division'])['T2_quality'].rank(method='dense', ascending=False).astype(int)

tourney_results['win'] = np.where(tourney_results['T1_Score'] > tourney_results['T2_Score'], 1, 0)

mean_win_ratio = pd.DataFrame({'seed_win_ratio': tourney_results.groupby('T1_seed')['win'].mean(),
                               'powerrank_win_ratio': tourney_results.groupby('T1_powerrank')['win'].mean()})
mean_win_ratio = mean_win_ratio.reset_index().rename(columns={'index':'seed'})
mean_win_ratio

In [ ]:
# シード情報とチームクオリティ、シード勝率の合体
TourneyCompactResults = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')

A_t1 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "TeamID1"})

A_t2 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "TeamID2"})
A_t1.index = A_t1.index.rename(['Season', 'TeamID'])
A_t2.index = A_t2.index.rename(['Season', 'TeamID'])
A_t2
A = A_t1.join(A_t2, how='outer').reset_index().drop(['TeamID1', 'TeamID2'], axis=1)
del A_t1, A_t2

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

l_seed = A.merge(seeds, on=['Season', 'TeamID'], how='left').drop('Seed', axis=1)
# l_seed = l_seed.drop('location_N', axis=1) # locationがリークさせていた→シンプルにそのトーナメントでの試合数になってしまうから
# del A, seeds
# l_seed['division_rank'] = l_seed.division.replace({'W':4, 'X':3, 'Y':2, 'Z':1})
# l_seed['seed_ratio'] = l_seed['seed'] * l_seed['division_rank']
# l_seed.drop(['seed', 'division', 'division_rank'], inplace=True, axis=1)
q_seed = l_seed.merge(team_quality, on=['Season', 'TeamID'], how='left').drop(['beta'], axis=1)\
        .merge(mean_win_ratio, on='seed', how='left')
q_seed = q_seed.drop(['seed_win_ratio', 'powerrank_win_ratio'], axis=1)
q_seed

# シード情報以外で回す場合はここから！

In [ ]:
# レギュラーシーズン試合結果からチーム毎の勝率を試合開催場所別に特徴量として作成
MRSCResults = pd.read_csv(DATA_DIR + '/MRegularSeasonCompactResults.csv')

A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)

H_l = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_H"})
N_l = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_N"})
A_l = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_A"})
lost = A_l.join(N_l, how='outer').join(H_l, how='outer').fillna(0)

win.index = win.index.rename(['Season', 'TeamID'])
lost.index = lost.index.rename(['Season', 'TeamID'])
wl = win.join(lost, how='outer').reset_index()
wl['win_pct_A'] = wl['win_A'] / (wl['win_A'] + wl['lost_A'])
wl['win_pct_N'] = wl['win_N'] / (wl['win_N'] + wl['lost_N'])
wl['win_pct_H'] = wl['win_H'] / (wl['win_H'] + wl['lost_H'])
wl['win_pct_All'] = (wl['win_A'] + wl['win_N'] + wl['win_H']) / \
    (wl['win_A'] + wl['win_N'] + wl['win_H'] + wl['lost_A']\
     + wl['lost_N'] + wl['lost_H'])

del A_w, N_w, H_w, H_l, N_l, A_l, win, lost

In [ ]:
# シーズン毎の勝利チームと敗北チームの得点差を、試合開催地別に特徴量として作成
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_N", "relScore": "relScore_N"})
H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_H", "relScore": "relScore_H"})
All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\
    .mean().rename(columns={"Score": "Score_All", "relScore": "relScore_All"})
scr = A_scr.join(N_scr, how='outer').join(H_scr, how='outer')\
    .join(All_scr, how='outer').fillna(0).reset_index()

del w_scr, l_scr, wl_scr, A_scr, H_scr, N_scr, All_scr

In [ ]:
# ここでチーム毎の試合データをもとに特徴量作成をやっている(1試合毎のやつ)
# 攻撃回数=2P試投数 + 3P試投数 + ターンオーバー数 + FT試投数 × 0.43)を用いて算出した。
# そして、総得点数を攻撃回数で除することで攻撃効率(Points Per Possession)を求めた。

MRSDetailedResults = pd.read_csv(DATA_DIR + '/MRegularSeasonDetailedResults.csv')
# 対戦相手のスコアを保持
MRSDetailedResults['WO_PTS'] = MRSDetailedResults.LScore
MRSDetailedResults['LO_PTS'] = MRSDetailedResults.WScore
# 対戦相手のオフェンスリバウンド回数、ディフェンスリバウンド率を保持
MRSDetailedResults['WO_DREB'] = MRSDetailedResults.LDR
MRSDetailedResults['LO_DREB'] = MRSDetailedResults.WDR
MRSDetailedResults['WO_OREB'] = MRSDetailedResults.LOR
MRSDetailedResults['LO_OREB'] = MRSDetailedResults.WOR
MRSDetailedResults['WO_REB'] = MRSDetailedResults.LDR + MRSDetailedResults.LOR
MRSDetailedResults['LO_REB'] = MRSDetailedResults.WDR + MRSDetailedResults.WOR
#対戦相手のFGA
MRSDetailedResults['WO_FGA'] = MRSDetailedResults.LFGA
MRSDetailedResults['LO_FGA'] = MRSDetailedResults.WFGA
# 対戦相手のFTA
MRSDetailedResults['WO_FTA'] = MRSDetailedResults.LFTA
MRSDetailedResults['LO_FTA'] = MRSDetailedResults.WFTA
# 対戦相手のTO
MRSDetailedResults['WO_TO'] = MRSDetailedResults.LTO
MRSDetailedResults['LO_TO'] = MRSDetailedResults.WTO

w = MRSDetailedResults.loc[:, ['Season', 'WScore', 'WTeamID', 'WFGM','WFGA','WFGM3'
                               ,'WFGA3','WFTM','WFTA','WOR','WDR','WAst',
                               'WTO','WStl','WBlk','WPF', 'WO_PTS', 'WO_DREB', 'WO_OREB', 'WO_REB',
                              'WO_FGA', 'WO_FTA', 'WO_TO']]
w.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']
l = MRSDetailedResults.loc[:, ['Season', 'LScore', 'LTeamID', 'LFGM','LFGA','LFGM3',
                               'LFGA3','LFTM','LFTA','LOR','LDR','LAst',
                               'LTO','LStl','LBlk','LPF', 'LO_PTS', 'LO_DREB', 'LO_OREB', 'LO_REB',
                              'LO_FGA', 'LO_FTA', 'LO_TO']]
l.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']

detail = pd.concat([w,l])
detail['goal_rate'] = detail.FGM / detail.FGA 
detail['3p_goal_rate'] = detail.FGM3 / detail.FGA3  
detail['ft_goal_rate'] = detail.FTM  / detail.FTA 
detail['total_attack'] = detail.FGA + detail.TO + detail.FTA * 0.43
detail['Points_Per_Possession'] = detail.Score / detail.total_attack
# https://bstatsplus.com/glossary このサイト参考にスタッツを計算
# 参考サイトその2:https://www.nogawanogawa.com/entry/basketball_analysis
# POSS:ポゼッション。攻撃回数。
detail['POSS'] = detail.FGA + 0.44 * detail.FTA + detail.TO - detail.OR
# ORTG:オフェンスレーティング。攻撃力。100POSSあたりの得点。
detail['ORTG'] = detail.Score / detail.POSS * 100
# DRTG:ディフェンスレーティング。守備力。100POSSあたりの失点。
detail['DRTG'] = detail.O_PTS / detail.POSS * 100
# NRTG:ネットレーティング。総合力。攻撃力と守備力の差でみる真の力。
detail['NRTG'] = detail.ORTG - detail.DRTG
# eFG%:得点効率。Effective FG%。「Four Factors」の1つ。3Pの価値は2Pの1.5倍という重みを付けて効率を評価。50%を超えるのが目安。
detail['eFG%'] = (detail.FGM + 0.5 * detail.FGM3) / detail.FGA
# PPT:シュート1本あたりの得点。得点期待値？Points Per Shot
detail['FGM2'] = detail.FGM - detail.FGM3 - detail.FTM
detail['PPS'] = (2 * detail.FGM2 + 3 * detail.FGM3) / detail.FGA
# 2PPS:2Pの得点期待値。1を超えるのが目安。
detail['FGA2'] = detail.FGA - detail.FGA3 - detail.FTA
detail['2PPS'] = 2 * detail.FGA2
# 3PPS:3Pの得点期待値。1を超えるのが目安。
detail['3PPS'] = 3 * detail['3p_goal_rate']
# TOV%:ターンオーバー率。「Four Factors」の1つ。1回の攻撃でどれだけTOVするか。少ないほうがよい。
detail['TOV%'] = detail.TO / detail.POSS
# OREB%:オフェンスリバウンド率。「Four Factors」の1つ。OREBの機会にどれだけ奪えるか。OREB機会は自チームのOREBと相手のDREBの合計。
detail['OREB%'] = detail.OR / (detail.OR + detail.O_DREB)
# DREB%:ディフェンスリバウンド率。DREBの機会にどれだけ奪えるか。DREB機会は自チームのDREBと相手のOREBの合計。
detail['DREB%'] = detail.DR / (detail.DR + detail.O_OREB)
# REB%:リバウンド率。リバウンド機会にどれだけ奪えるか。
detail['REB'] = detail.DR + detail.OR
# REB = detail.DR + detail.OR
detail['REB%'] = (detail.REB) / (detail.REB + detail.O_REB)
# FTR:フリースローレーティング。「Four Factors」の1つ。フリースローをどれだけ獲得できるか。アタックできてるかどうかの指標。
detail['FTR'] = detail.FTA / detail.FGA
# TS%:True Shooting Percentage。真のシュート成功率。eFG%がフィールドゴールだけなのに対して、TS%はFTも加味した得点効率。
detail['TS%'] = 0.5 * detail.Score / (detail.FGA + 0.44 * detail.FTA)
# 3P/2P:3PAと2PAの比率。アウトサイドとインサイドのどちらに重きを置いているかを測る指標。
detail['3P/2P'] = detail.FGA3 / detail.FGA2
# AST%:アシスト率。アシストによるフィールドゴールがどれだけあるかを測る指標。
detail['AST%'] = detail.Ast / detail.FGM
# AST/TO:ASTとTOVの比率。アシストが多くターンオーバーが少ないのが優秀とみることが多い。
detail['AST/TO'] = detail.Ast / detail.TO
# PACE:ペース。試合展開の速さの目安。1試合＝40分あたりの攻撃回数。
detail['PACE'] = detail.POSS * 5 / 200
# 以下小川さんの作った特徴量
#対戦相手のPOSS
detail['O_POSS'] = detail.O_FGA + 0.44 * detail.O_FTA + detail.O_TO - detail.O_OREB
#LO_DRTG:対戦相手のディフェンスレーティング
detail['O_DRTG'] = detail.Score / detail.O_POSS * 100
#AdjEM 	自チームのオフェンス　ー　対戦相手のディフェンス
detail['AdjEM'] = detail.ORTG - detail.O_DRTG


# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         '2PPS', '3PPS', 'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%', '3P/2P',
#                                         'AST%', 'AST/TO', 'PACE', 'AdjEM', 'REB', 'REB%']\
#                                         .mean().fillna(0).reset_index()

dt = detail.groupby(['Season','TeamID'])['Points_Per_Possession', 'NRTG', 'PACE', 'TS%', 'AdjEM',
                                         'eFG%', 'TOV%', 'OREB%', 'DREB%', 'FTR']\
                                        .mean().fillna(0).reset_index()

# dt.drop(['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF'], axis=1, inplace=True)
# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%', '3P/2P',
#                                         'AST%', 'AST/TO', 'PACE', '2PPS', '3PPS']\
#                                         .mean().fillna(0).reset_index()

del w, l

In [ ]:
# 個人が集計したチームスコア・ランキングを基に、シーズン○節毎のチームスコア・ランキング推移を作成
MMOrdinals = pd.read_csv(DATA_DIR + '/MMasseyOrdinals.csv')

MOR_127_128 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 127) \
                 | (MMOrdinals.RankingDayNum == 128))]\
                [['Season','TeamID','OrdinalRank']]
MOR_50_51 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 50) \
                 | (MMOrdinals.RankingDayNum == 51))]\
                [['Season','TeamID','OrdinalRank']]
MOR_15_16 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 15) \
                 | (MMOrdinals.RankingDayNum == 16))]\
                [['Season','TeamID','OrdinalRank']]

MOR_127_128 = MOR_127_128.rename(columns={'OrdinalRank':'OrdinalRank_127_128'})
MOR_50_51 = MOR_50_51.rename(columns={'OrdinalRank':'OrdinalRank_50_51'})
MOR_15_16 = MOR_15_16.rename(columns={'OrdinalRank':'OrdinalRank_15_16'})

MOR = MOR_127_128.merge(MOR_50_51, how='left', on=['Season','TeamID'])\
        .merge(MOR_15_16, how='left', on=['Season','TeamID'])

## normalizing Rank values by its season maxium as it varies by seasons
MOR_max = MOR.groupby('Season')['OrdinalRank_127_128','OrdinalRank_50_51',
                                'OrdinalRank_15_16'].max().reset_index()
MOR_max.columns = ['Season', 'maxRank_127_128', 'maxRank_50_51', 'maxRank_15_16']

MOR_tmp = MMOrdinals[(MMOrdinals.SystemName == 'MOR') \
                     & (MMOrdinals.RankingDayNum < 133)]
MOR_stats = MOR_tmp.groupby(['Season','TeamID'])['OrdinalRank']\
            .agg(['max','min','std','mean']).reset_index()
MOR_stats.columns = ['Season','TeamID','RankMax','RankMin','RankStd','RankMean']

MOR = MOR.merge(MOR_max, how='left', on='Season')\
        .merge(MOR_stats, how='left', on=['Season','TeamID'])
MOR['OrdinalRank_127_128'] = MOR['OrdinalRank_127_128'] / MOR['maxRank_127_128']
MOR['OrdinalRank_50_51'] = MOR['OrdinalRank_50_51'] / MOR['maxRank_50_51']
MOR['OrdinalRank_15_16'] = MOR['OrdinalRank_15_16'] / MOR['maxRank_15_16']
MOR['RankTrans_50_51_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_50_51']
MOR['RankTrans_15_16_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_15_16']

# MOR['RankMax'] = MOR['RankMax'] / MOR['maxRank_127_128']
# MOR['RankMin'] = MOR['RankMin'] / MOR['maxRank_127_128']
# MOR['RankStd'] = MOR['RankStd'] / MOR['maxRank_127_128']
# MOR['RankMean'] = MOR['RankMean'] / MOR['maxRank_127_128']

MOR.drop(['OrdinalRank_50_51','OrdinalRank_15_16', 'maxRank_50_51'
          ,'maxRank_15_16'],axis=1, inplace=True)

del MOR_127_128, MOR_50_51, MOR_15_16, MOR_max, MOR_tmp, MOR_stats

In [ ]:
# 作成したデータフレームのカラム名を後のマージの為にリネーム
wl_1 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_1.columns ]

wl_2 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_2.columns ]

scr_1 = scr.copy()
scr_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_1.columns ]

scr_2 = scr.copy()
scr_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_2.columns ]

dt_1 = dt.copy()
dt_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_1.columns ]

dt_2 = dt.copy()
dt_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_2.columns ]

MOR_1 = MOR.copy()
MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_1.columns ]

MOR_2 = MOR.copy()
MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_2.columns ]

q_seed_1 = q_seed.copy()
q_seed_1.columns = [str(col) + '_1' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_1.columns]

q_seed_2 = q_seed.copy()
q_seed_2.columns = [str(col) + '_2' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_2.columns]

In [ ]:
# ここまでで作成したデータフレームをマージする為の基のデータフレーム（レギュラーシーズンの試合結果）作成
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(np.int64)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(np.int64)
tourney1 = regular_results.loc[:, ['Season','T1_TeamID','T2_TeamID', 'T1_win']]
tourney1.columns = ['Season','TeamID1','TeamID2', 'result']

tourney2 = regular_results.loc[:, ['Season','T2_TeamID','T1_TeamID', 'T2_win']]
tourney2.columns = ['Season','TeamID1','TeamID2', 'result']

regular_final = pd.concat([tourney1, tourney2])
del tourney1, tourney2
regular_final

In [ ]:
# データをマージする関数
def merge_data(df):

#     df = df.merge(wl_1, how='left', left_on=['Season','TeamID1'],
#                   right_on=['Season','TeamID'])
#     df = df.merge(wl_2, how='left', left_on=['Season','TeamID2'],
#                   right_on=['Season','TeamID'])
#     df = df.drop(['TeamID_x','TeamID_y'], axis=1)


#     df = df.merge(scr_1, how='left', left_on=['Season','TeamID1'],
#                   right_on=['Season','TeamID'])
#     df = df.merge(scr_2, how='left', left_on=['Season','TeamID2'],
#                   right_on=['Season','TeamID'])
#     df = df.drop(['TeamID_x','TeamID_y'], axis=1)

#     df['win_pct_A_diff'] = df['win_pct_A_1'] - df['win_pct_A_2']
#     df['win_pct_N_diff'] = df['win_pct_N_1'] - df['win_pct_N_2']
#     df['win_pct_H_diff'] = df['win_pct_H_1'] - df['win_pct_H_2']
#     df['win_pct_All_diff'] = df['win_pct_All_1'] - df['win_pct_All_2']

#     df['Score_A_diff'] = df['Score_A_1'] - df['Score_A_2']
#     df['Score_N_diff'] = df['Score_N_1'] - df['Score_N_2']
#     df['Score_H_diff'] = df['Score_H_1'] - df['Score_H_2']
#     df['Score_All_diff'] = df['Score_All_1'] - df['Score_All_2']

#     df['relScore_A_diff'] = df['relScore_A_1'] - df['relScore_A_2']
#     df['relScore_N_diff'] = df['relScore_N_1'] - df['relScore_N_2']
#     df['relScore_H_diff'] = df['relScore_H_1'] - df['relScore_H_2']
#     df['relScore_All_diff'] = df['relScore_All_1'] - df['relScore_All_2']

    df = df.merge(dt_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(dt_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(MOR_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(MOR_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)
    
    df = df.merge(q_seed_1, how='left', left_on=['Season', 'TeamID1'],
                 right_on=['Season', 'TeamID'])
    df = df.merge(q_seed_2, how='left', left_on=['Season', 'TeamID2'],
                 right_on=['Season', 'TeamID'])
    df = df.drop(['TeamID_x', 'TeamID_y'], axis=1)

#     df['OrdinalRank_127_128_diff'] = df['OrdinalRank_127_128_1'] \
#         - df['OrdinalRank_127_128_2']

    df = df.fillna(-1)
    
    for col in df.columns:
        if (df[col] == np.inf).any() or (df[col] == -np.inf).any():
            df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
    
    return df

# 以下モデルに学習させるtrainデータを作成
final_regular_result = merge_data(regular_final)
final_regular_result = final_regular_result.loc[final_regular_result.Season >= 2015,:].reset_index(drop=True)

if STAGE_1:
    final_regular_result = final_regular_result.loc[final_regular_result.Season < 2022, :]

In [ ]:
# submissionデータから、実際にモデルに予測させる試合の組み合わせ表（各チームスタッツデータ込み）の作成
if STAGE_1:
    MSampleSubmission = pd.read_csv(DATA_DIR + '/MSampleSubmissionStage2.csv')
else:
    MSampleSubmission = pd.read_csv(DATA_DIR + None) # put stage 2 submission file link here

test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1['TeamID1'] = test1.ID.apply(lambda x: int(x[5:9]))
test1['TeamID2'] = test1.ID.apply(lambda x: int(x[10:14]))

test2 = MSampleSubmission.copy()
test2['Season'] = test2.ID.apply(lambda x: int(x[0:4]))
test2['TeamID1'] = test2.ID.apply(lambda x: int(x[10:14]))
test2['TeamID2'] = test2.ID.apply(lambda x: int(x[5:9]))

test = pd.concat([test1,test2]).drop(['Pred'], axis=1)
test = merge_data(test)

In [ ]:
# 作成したtrainデータ、testデータをcsv化
final_regular_result.to_csv('final_regular_result.csv', index=False)
test.to_csv('stage1_test.csv', index=False)